In [1]:
# Import dependencies
import pandas as pd
import numpy as np

import tensorflow
tensorflow.keras.__version__

'2.3.0-tf'

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Import Dataset

In [3]:
nineFive_random_df = pd.read_csv('dataset/95k-random.csv')
nineFive_random_df.head(5)

,id,account_age,no_follower,no_following,no_userfavourites,no_lists,no_tweets,no_retweets,no_hashtag,no_usermention,no_urls,no_char,no_digits,class
0,1,1235,12,31,0,0,108,0,1,0,1,30,0,spammer
1,2,695,126,569,16,0,5048,1,1,2,1,109,1,spammer
2,3,448,4,63,0,1,860,0,0,0,1,36,4,spammer
3,4,1322,8,294,2,0,237,0,0,0,1,35,2,spammer
4,5,111,1461,1365,438,10,6894,0,1,0,1,55,6,spammer


In [4]:
# nineFive_random_df = nineFive_random_df.drop('id', axis=1)
nineFive_clean = nineFive_random_df[['account_age', 'no_follower', 'no_following', 'no_tweets', 'no_retweets', 'class']]
nineFive_clean.head()

,account_age,no_follower,no_following,no_tweets,no_retweets,class
0,1235,12,31,108,0,spammer
1,695,126,569,5048,1,spammer
2,448,4,63,860,0,spammer
3,1322,8,294,237,0,spammer
4,111,1461,1365,6894,0,spammer


# Data Pre-Processing

In [5]:
# nineFive_random_df['class'].value_counts()
nineFive_clean['class'].value_counts()

non-spammer    95000
spammer         5000
Name: class, dtype: int64

In [6]:
# X = nineFive_random_df.drop('class', axis=1)
# y = nineFive_random_df['class']
# print(X.shape, y.shape)
X = nineFive_clean.drop('class', axis=1)
y = nineFive_clean['class']
print(X.shape, y.shape)

(100000, 5) (100000,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model, add layers & hidden nodes
model = Sequential()
model.add(Dense(units=15, activation='relu', input_dim=5))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=5, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                90        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 12        
Total params: 317
Trainable params: 317
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
2344/2344 - 10s - loss: 0.2147 - accuracy: 0.9499
Epoch 2/100
2344/2344 - 10s - loss: 0.1954 - accuracy: 0.9499
Epoch 3/100
2344/2344 - 9s - loss: 0.1945 - accuracy: 0.9500
Epoch 4/100
2344/2344 - 8s - loss: 0.1944 - accuracy: 0.9500
Epoch 5/100
2344/2344 - 9s - loss: 0.1941 - accuracy: 0.9501
Epoch 6/100
2344/2344 - 9s - loss: 0.1939 - accuracy: 0.9500
Epoch 7/100
2344/2344 - 9s - loss: 0.1939 - accuracy: 0.9500
Epoch 8/100
2344/2344 - 11s - loss: 0.1938 - accuracy: 0.9501
Epoch 9/100
2344/2344 - 10s - loss: 0.1936 - accuracy: 0.9501
Epoch 10/100
2344/2344 - 9s - loss: 0.1934 - accuracy: 0.9501
Epoch 11/100
2344/2344 - 9s - loss: 0.1933 - accuracy: 0.9501
Epoch 12/100
2344/2344 - 12s - loss: 0.1932 - accuracy: 0.9501
Epoch 13/100
2344/2344 - 9s - loss: 0.1931 - accuracy: 0.9501
Epoch 14/100
2344/2344 - 10s - loss: 0.1931 - accuracy: 0.9501
Epoch 15/100
2344/2344 - 12s - loss: 0.1929 - accuracy: 0.9501
Epoch 16/100
2344/2344 - 9s - loss: 0.1927 - accuracy: 0.9501
Epoch 17/1

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 - 2s - loss: 0.1837 - accuracy: 0.9504
Loss: 0.18365322053432465, Accuracy: 0.9503999948501587


In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:100])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [18]:
predict = model.predict(X_test_scaled[:100])
labels = np.argmax(predict, axis=1)
prediction_labels = label_encoder.inverse_transform(labels)

In [19]:
print(f"Predicted Classes: {prediction_labels}")
print(f"Actual labels: {list(y_test[:100])}")

Predicted Classes: ['non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' '

In [20]:
# import joblib
# joblib.dump(scaler,"bot_scaler.pkl")

In [21]:
model.save("bot_trained.h5")